## 数据库
找到每个期货品种每天对应的主力合约，次主力合约已经复权因子

In [1]:
import pandas as pd
from datetime import datetime, timedelta

In [2]:
test_rawdata = pd.read_csv('test.csv') 
futures_rawdata = pd.read_csv('futures_description.csv')
# 合并
rawdata = test_rawdata.merge(futures_rawdata, how = 'inner', left_on = 'CONTRACT', right_on = 'symbol').sort_values(by = 'DATE',ascending=False)

In [3]:
rawdata

,close,COMMODITY,CONTRACT,DATE,OPENINT,SETTLEMENT,code,symbol,maturity_date,start_date
334476,3088.8,IH,IH2002,2020-01-03,1683,3091.6,IH,IH2002,2020-02-21,2019-12-23
125908,7739.0,OI,OI2005,2020-01-03,115337,7758.0,OI,OI2005,2020-05-19,2019-05-20
329492,547.2,ZC,ZC2008,2020-01-03,1,547.2,ZC,ZC2008,2020-08-07,2019-08-08
329392,549.2,ZC,ZC2007,2020-01-03,7972,550.2,ZC,ZC2007,2020-07-07,2019-07-08
158651,2329.0,RM,RM2005,2020-01-03,262382,2335.0,RM,RM2005,2020-05-19,2019-05-20
...,...,...,...,...,...,...,...,...,...,...
3881,15745.0,CF,CF0609,2006-01-05,830,15700.0,CF,CF0609,2006-09-14,2005-10-10
3733,15580.0,CF,CF0608,2006-01-05,1622,15555.0,CF,CF0608,2006-08-14,2005-09-01
3606,15540.0,CF,CF0607,2006-01-05,2094,15495.0,CF,CF0607,2006-07-14,2005-08-01
3501,15480.0,CF,CF0606,2006-01-05,5922,15440.0,CF,CF0606,2006-06-14,2005-07-01


# 主力合约
找到每个期货品种每天对应的主力合约，次主力合约以及复权因子
1.  连续3个交易日每天持仓量均为最大的合约，在第4个交易日成为主力合约
2. 如果某主力合约在到期日仍然为持仓量最大，在剩余该品种合约中在最近3个交易日内，累计持仓量最大的合约作为下一个交易日的主力合约

### 例子, 期货品种 = AP

In [7]:
part_data = rawdata.loc[rawdata.code == 'AP']
starttime, endtime = part_data.sort_values(by = 'DATE').iloc[0,3], part_data.sort_values(by = 'DATE').iloc[-1,3] #这个期货品种的起始时间和结束时间

# 按照开始时间和结束时间，创建时间列表。
day_gap = (datetime.strptime(endtime, "%Y-%m-%d") - datetime.strptime(starttime, "%Y-%m-%d")).days
day_list = [(datetime.strptime(endtime, "%Y-%m-%d") - timedelta(days = i)).strftime("%Y-%m-%d") for i in range(0,day_gap)]

# 创建新的dataframe 填充时间列表和期货种类
part_df = pd.DataFrame(columns = ['date', 'code'])
part_df['date'] = day_list 
part_df['code'] = 'AP'
part_df 

,date,code
0,2020-01-03,AP
1,2020-01-02,AP
2,2020-01-01,AP
3,2019-12-31,AP
4,2019-12-30,AP
...,...,...
737,2017-12-27,AP
738,2017-12-26,AP
739,2017-12-25,AP
740,2017-12-24,AP


接下来得任务是填充part_df 表格，找到每个时间的主力合约次主力合约

In [8]:
def get_max_openint(x): 
    '''输出持仓量最大的合约'''
    contract = part_data.loc[part_data.DATE == x].sort_values(by = 'OPENINT', ascending=True)
    if not contract.empty : 
        return contract.iloc[0,2]

part_df['max_openint'] = part_df.date.apply(lambda x: get_max_openint(x))

可以看到有许多空值，现在暂时对空值处理。需要在最大持仓量的合约中找到连续三天持仓的合约， 使其在第4个交易日成为主力合约

In [9]:
max_openint_list = list(part_df.max_openint)

for i in range(len(max_openint_list), 2, -1):
    j = i-3
    a_list = max_openint_list[j:i]
    if a_list[0] == a_list[1] == a_list[2]: 
        part_df.loc[part_df.index == j-1, 'dom_contract'] = a_list[2]
part_df

,date,code,max_openint,dom_contract
0,2020-01-03,AP,AP2001,NaN
1,2020-01-02,AP,AP2001,NaN
2,2020-01-01,AP,None,NaN
3,2019-12-31,AP,AP2001,NaN
4,2019-12-30,AP,AP2012,NaN
...,...,...,...,...
737,2017-12-27,AP,AP1812,NaN
738,2017-12-26,AP,AP1812,NaN
739,2017-12-25,AP,AP1812,NaN
740,2017-12-24,AP,None,NaN


如果某主力合约在到期日仍然为持仓量最大，在剩余该品种合约中在最近3个交易日内，累计持仓量最大的合约作为下一个交易日的主力合约

In [10]:
aa = part_data[['CONTRACT', 'maturity_date']].drop_duplicates() #记录到期时间
part_df2 = part_df.merge(aa, how = 'left', left_on = 'dom_contract', right_on = 'CONTRACT')
change_dom = part_df2.loc[(part_df2.maturity_date.notnull()) & (part_df2.date == part_df2.maturity_date)] #需要转化主力合约的表格

In [31]:
def get_max_openint2(x, feature):
    '''3天内累计持仓量最大的合约'''
    starttime, endtime =(datetime.strptime(x.date, "%Y-%m-%d") - timedelta(days = 2)).strftime("%Y-%m-%d"),(datetime.strptime(x.date, "%Y-%m-%d") + timedelta(days = 1)).strftime("%Y-%m-%d")
    # 计算时间内累计最大持仓量的合约
    sum_3day = part_data.loc[(part_data.CONTRACT != x[feature])&(part_data.DATE < endtime)&(part_data.DATE >= starttime)].groupby(by = 'CONTRACT')['OPENINT'].sum().to_frame().reset_index()
    #输出合约名字
    return sum_3day.sort_values(by = 'OPENINT', ascending=False).iloc[0, 0] 

if not change_dom.empty: # 有可能没有需要转化主力的合约
    # 更新需要转化主力合约的表格
    change_dom['dom_contract'] = change_dom.apply(lambda x: get_max_openint2(x, 'max_openint'), axis = 1)

    #更新part_df应该输出的表格中的主力合约
    part_df.loc[part_df.index.isin(list(change_dom.index + 1)), 'dom_contract'] = change_dom.dom_contract
    
part_df

,date,code,max_openint,dom_contract
0,2020-01-03,IH,IH2002,NaN
1,2020-01-02,IH,IH2002,NaN
2,2020-01-01,IH,None,NaN
3,2019-12-31,IH,IH2002,NaN
4,2019-12-30,IH,IH2002,NaN
...,...,...,...,...
1718,2015-04-21,IH,IH1512,NaN
1719,2015-04-20,IH,IH1512,NaN
1720,2015-04-19,IH,None,NaN
1721,2015-04-18,IH,None,NaN


现在看到有许多空值情况。可以做个总结：有些日期没有主力合约因为-
1. 日期数据缺失。有些日期并没有记录当天的持仓量
2. 首尾缺失。主力合约最早从第4日开始，所以前三日没有主力合约

In [12]:
# 输出主力合约表格
output_part_df = part_df.loc[part_df.dom_contract.notnull(), ['date', 'code', 'dom_contract']]
output_part_df

,date,code,dom_contract
6,2019-12-28,AP,AP2012
7,2019-12-27,AP,AP2012
8,2019-12-26,AP,AP2012
13,2019-12-21,AP,AP2012
14,2019-12-20,AP,AP2012
...,...,...,...
727,2018-01-06,AP,AP1812
728,2018-01-05,AP,AP1812
734,2017-12-30,AP,AP1812
735,2017-12-29,AP,AP1812


### 脚本汇总整理

In [33]:
def get_dom(code):
    '''对每个期货品种，计算主力合约'''
    part_data = rawdata.loc[rawdata.code == code]
    starttime, endtime = part_data.sort_values(by = 'DATE').iloc[0,3], part_data.sort_values(by = 'DATE').iloc[-1,3] #这个期货品种的起始时间和结束时间

    # 按照开始时间和结束时间，创建时间列表。
    day_gap = (datetime.strptime(endtime, "%Y-%m-%d") - datetime.strptime(starttime, "%Y-%m-%d")).days
    day_list = [(datetime.strptime(endtime, "%Y-%m-%d") - timedelta(days = i)).strftime("%Y-%m-%d") for i in range(0,day_gap)]

    # 创建新的dataframe 填充时间列表和期货种类
    part_df = pd.DataFrame(columns = ['date', 'code'])
    part_df['date'] = day_list 
    part_df['code'] = code
    
    # 接下来得任务是填充part_df 表格，找到每个时间的主力合约次主力合约
    def get_max_openint(x): 
        '''输出持仓量最大的合约'''
        contract = part_data.loc[part_data.DATE == x].sort_values(by = 'OPENINT', ascending=True)
        if not contract.empty : 
            return contract.iloc[0,2]

    part_df['max_openint'] = part_df.date.apply(lambda x: get_max_openint(x))
    
    #可以看到有许多空值，现在暂时对空值处理。需要在最大持仓量的合约中找到连续三天持仓的合约， 使其在第4个交易日成为主力合约
    max_openint_list = list(part_df.max_openint)

    for i in range(len(max_openint_list), 2, -1):
        j = i-3
        a_list = max_openint_list[j:i]
        if a_list[0] == a_list[1] == a_list[2]: 
            part_df.loc[part_df.index == j-1, 'dom_contract'] = a_list[2]
            
    # 如果某主力合约在到期日仍然为持仓量最大，在剩余该品种合约中在最近3个交易日内，累计持仓量最大的合约作为下一个交易日的主力合约
    aa = part_data[['CONTRACT', 'maturity_date']].drop_duplicates() #记录到期时间
    part_df2 = part_df.merge(aa, how = 'left', left_on = 'dom_contract', right_on = 'CONTRACT')
    change_dom = part_df2.loc[(part_df2.maturity_date.notnull()) & (part_df2.date == part_df2.maturity_date)] #需要转化主力合约的表格
    
    def get_max_openint2(x, feature):
        '''3天内累计持仓量最大的合约'''
        starttime, endtime =(datetime.strptime(x.date, "%Y-%m-%d") - timedelta(days = 2)).strftime("%Y-%m-%d"),(datetime.strptime(x.date, "%Y-%m-%d") + timedelta(days = 1)).strftime("%Y-%m-%d")
        # 计算时间内累计最大持仓量的合约
        sum_3day = part_data.loc[(part_data.CONTRACT != x[feature])&(part_data.DATE < endtime)&(part_data.DATE >= starttime)].groupby(by = 'CONTRACT')['OPENINT'].sum().to_frame().reset_index()
        #输出合约名字
        return sum_3day.sort_values(by = 'OPENINT', ascending=False).iloc[0, 0] 
    
    if not change_dom.empty: # 有可能没有需要转化主力的合约
        # 更新需要转化主力合约的表格
        change_dom['dom_contract'] = change_dom.apply(lambda x: get_max_openint2(x, 'max_openint'), axis = 1)

        #更新part_df应该输出的表格中的主力合约
        part_df.loc[part_df.index.isin(list(change_dom.index+1)), 'dom_contract'] = change_dom.dom_contract
    
    # 输出主力合约表格
    output_part_df = part_df.loc[part_df.dom_contract.notnull(), ['date', 'code', 'dom_contract']].reset_index(drop = True)
    
    return output_part_df

get_dom('AP')

/tmp/ipykernel_2449/1205973151.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  change_dom['dom_contract'] = change_dom.apply(lambda x: get_max_openint2(x, 'max_openint'), axis = 1)


,date,code,dom_contract
0,2019-12-28,AP,AP2012
1,2019-12-27,AP,AP2012
2,2019-12-26,AP,AP2012
3,2019-12-21,AP,AP2012
4,2019-12-20,AP,AP2012
...,...,...,...
248,2018-01-06,AP,AP1812
249,2018-01-05,AP,AP1812
250,2017-12-30,AP,AP1812
251,2017-12-29,AP,AP1812


# 次主力合约
1. 次主力合约是除去了该品种（新）主力合约以外，在新主力合约满足换月条件的3个交易日内，累计持仓量最大的合约
2. 次主力合约随着主力合约的切换而切换

### 例子 AP

In [14]:
# 首先找到转换主力合约的次主力合约
output_part_df = get_dom('AP')
switchdom_df = output_part_df.drop_duplicates(subset = 'dom_contract', keep = "last") 

def get_max_openint3(x, feature):
    '''发生前三天内累计持仓量最大的合约， 和get_max_openint2 区别是时间不同'''
    starttime, endtime =(datetime.strptime(x.date, "%Y-%m-%d") - timedelta(days = 3)).strftime("%Y-%m-%d"), x.date
    # 计算时间内累计最大持仓量的合约
    sum_3day = part_data.loc[(part_data.CONTRACT != x[feature])&(part_data.DATE < endtime)&(part_data.DATE >= starttime)].groupby(by = 'CONTRACT')['OPENINT'].sum().to_frame().reset_index()
    #输出合约名字
    return sum_3day.sort_values(by = 'OPENINT', ascending=False).iloc[0, 0] 

# 添加字段 次主力合约
switchdom_df['subdom_contract'] = switchdom_df.apply(lambda x: get_max_openint3(x, 'dom_contract'), axis = 1)

# 更新到 output_part_df 
output_part_df2 = switchdom_df.merge(output_part_df.dom_contract, how = 'left', on = 'dom_contract')
output_part_df2

/tmp/ipykernel_2449/3604058620.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  change_dom['dom_contract'] = change_dom.apply(lambda x: get_max_openint2(x, 'max_openint'), axis = 1)
/tmp/ipykernel_2449/3968130400.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  switchdom_df['subdom_contract'] = switchdom_df.apply(lambda x: get_max_openint3(x, 'dom_contract'), axis = 1)


,date,code,dom_contract,subdom_contract
0,2019-12-19,AP,AP2012,AP2005
1,2019-12-19,AP,AP2012,AP2005
2,2019-12-19,AP,AP2012,AP2005
3,2019-12-19,AP,AP2012,AP2005
4,2019-12-19,AP,AP2012,AP2005
...,...,...,...,...
248,2017-12-28,AP,AP1812,AP1805
249,2017-12-28,AP,AP1812,AP1805
250,2017-12-28,AP,AP1812,AP1805
251,2017-12-28,AP,AP1812,AP1805


### 脚本汇总

In [15]:
def get_subdom(code):
    '''输出次主力合约'''
    output_part_df = get_dom(code)
    # 首先找到转换主力合约的次主力合约
    switchdom_df = output_part_df.drop_duplicates(subset = 'dom_contract', keep = "last")
    
    def get_max_openint3(x, feature):
        '''发生前三天内累计持仓量最大的合约， 和get_max_openint2 区别是时间不同'''
        starttime, endtime =(datetime.strptime(x.date, "%Y-%m-%d") - timedelta(days = 3)).strftime("%Y-%m-%d"), x.date
        # 计算时间内累计最大持仓量的合约
        sum_3day = part_data.loc[(part_data.CONTRACT != x[feature])&(part_data.DATE < endtime)&(part_data.DATE >= starttime)].groupby(by = 'CONTRACT')['OPENINT'].sum().to_frame().reset_index()
        #输出合约名字
        return sum_3day.sort_values(by = 'OPENINT', ascending=False).iloc[0, 0] 

    # 添加字段 次主力合约
    switchdom_df['subdom_contract'] = switchdom_df.apply(lambda x: get_max_openint3(x, 'dom_contract'), axis = 1)

    # 更新到 output_part_df 
    output_part_df2 = switchdom_df.merge(output_part_df.dom_contract, how = 'left', on = 'dom_contract')
    
    return output_part_df2[['date', 'code', 'subdom_contract']]

get_subdom('AP')

/tmp/ipykernel_2449/3604058620.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  change_dom['dom_contract'] = change_dom.apply(lambda x: get_max_openint2(x, 'max_openint'), axis = 1)
/tmp/ipykernel_2449/1973389626.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  switchdom_df['subdom_contract'] = switchdom_df.apply(lambda x: get_max_openint3(x, 'dom_contract'), axis = 1)


,date,code,subdom_contract
0,2019-12-19,AP,AP2005
1,2019-12-19,AP,AP2005
2,2019-12-19,AP,AP2005
3,2019-12-19,AP,AP2005
4,2019-12-19,AP,AP2005
...,...,...,...
248,2017-12-28,AP,AP1805
249,2017-12-28,AP,AP1805
250,2017-12-28,AP,AP1805
251,2017-12-28,AP,AP1805


# 复权因子计算
1. 主力合约未切换，复权因子（收盘和结算）与前值保持不变, 初始复权因子为1
2. 主力合约发成切换时，
收盘价复权因子：

f_actorclose_t = f_actorclose_t-1 * (close_old_t-1 / pre_close_new_t)
其中close_old_t-1 为旧主力合约t-1天（前一天）的收盘价，pre_close_new_t表示新主力合约t天（当天）的上一个交易日收盘价

## 主力合约

### 例子 AP

In [16]:
part_dom = get_dom('AP')# 获得主力合约
#从rawdata获得信息并且合并
part_actorclose_dom = part_dom.merge(rawdata[['DATE','CONTRACT','OPENINT','SETTLEMENT','symbol', 'close', ]], how = 'left', left_on = ['date','dom_contract'], right_on = ['DATE', 'CONTRACT'])
part_actorclose_dom

/tmp/ipykernel_2449/3604058620.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  change_dom['dom_contract'] = change_dom.apply(lambda x: get_max_openint2(x, 'max_openint'), axis = 1)


,date,code,dom_contract,DATE,CONTRACT,OPENINT,SETTLEMENT,symbol,close
0,2019-12-28,AP,AP2012,NaN,NaN,NaN,NaN,NaN,NaN
1,2019-12-27,AP,AP2012,2019-12-27,AP2012,4698.0,7700.0,AP2012,7669.0
2,2019-12-26,AP,AP2012,2019-12-26,AP2012,4686.0,7694.0,AP2012,7706.0
3,2019-12-21,AP,AP2012,NaN,NaN,NaN,NaN,NaN,NaN
4,2019-12-20,AP,AP2012,2019-12-20,AP2012,3000.0,7755.0,AP2012,7736.0
...,...,...,...,...,...,...,...,...,...
248,2018-01-06,AP,AP1812,NaN,NaN,NaN,NaN,NaN,NaN
249,2018-01-05,AP,AP1812,2018-01-05,AP1812,308.0,7225.0,AP1812,7230.0
250,2017-12-30,AP,AP1812,NaN,NaN,NaN,NaN,NaN,NaN
251,2017-12-29,AP,AP1812,2017-12-29,AP1812,450.0,7177.0,AP1812,7179.0


需要新增三个字段：pre_close 上一个交易日收盘价, pre_settlement上一个交易日结算价，next_symbol 下一个交易日期货合约

In [17]:
def get_pre_close(x,feature):
    # 上一个交易日日期
    pre_date = (datetime.strptime(x.date, "%Y-%m-%d") - timedelta(days = 1)).strftime("%Y-%m-%d")
    part_data1 = part_data.loc[(part_data.CONTRACT == x[feature])&(part_data.DATE == pre_date)]
    
    if not part_data1.empty: 
        return part_data1.close.values[0]
def get_pre_settlement(x, feature):
    # 上一个交易日日期
    pre_date = (datetime.strptime(x.date, "%Y-%m-%d") - timedelta(days = 1)).strftime("%Y-%m-%d")
    part_data1 = part_data.loc[(part_data.CONTRACT == x[feature])&(part_data.DATE == pre_date)]
    
    if not part_data1.empty: 
        return part_data1.SETTLEMENT.values[0]
    
def get_next_symbol(x, feature):
    # 下一个交易日期
    next_date = (datetime.strptime(x.date, "%Y-%m-%d") + timedelta(days = 1)).strftime("%Y-%m-%d")
    part_data2 = part_data.loc[(part_data.CONTRACT == x[feature])&(part_data.DATE == next_date)]
    
    if not part_data2.empty: 
        return part_data2.symbol.values[0]
    
# 新增三个字段
part_actorclose_dom['pre_close'] = part_actorclose_dom.apply(lambda x: get_pre_close(x, 'dom_contract'), axis = 1) 
part_actorclose_dom['pre_settlement'] =  part_actorclose_dom.apply(lambda x: get_pre_settlement(x, 'dom_contract'), axis = 1) 
part_actorclose_dom['next_symbol'] =  part_actorclose_dom.apply(lambda x: get_next_symbol(x, 'dom_contract'), axis = 1) 

 收盘价复权因子:
 1. f_actorclose_t = f_actorclose_t-1 * (close_old_t-1 / pre_close_new_t)
 2. f_actorsettlement_t = f_actorsettlement_t-1 * (settlement_old_t-1 / pre_settlement_new_t)

In [20]:
def get_factors(df, feature):
    '''计算复权因子factor_close & factor_settlement， 计算方式一致'''
    df_change_old = df.drop_duplicates(subset = feature, keep = 'first') #旧主力的最后一个交易日
    df_change_new = df.drop_duplicates(subset = feature, keep = 'last') # 新主力的第一个交易日
    
    # 初始复权因子 = 1
    f_actorclose = 1
    f_actorsettlement = 1
    for i in range(len(list(df_change_old.index))-1, 0, -1):
        # 首先更新呢表格
        t_1_index = list(df_change_old.index)[i]
        df_change_old.loc[df_change_old.index == t_1_index, 'factor_close'] = f_actorclose
        df_change_old.loc[df_change_old.index == t_1_index, 'factor_settlement'] = f_actorsettlement
        #旧主力合约t-1天（前一天）的收盘价
        close_old_t_1 = df_change_old.loc[df_change_old.index == t_1_index].pre_close.values[0]
        settlement_old_t_1 = df_change_old.loc[df_change_old.index == t_1_index].pre_settlement.values[0]
        # 表示新主力合约t天（当天）的上一个交易日收盘价
        t_index = list(df_change_old.index)[i-1]
        t_dom_contract = df_change_old.loc[df_change_old.index == t_index][feature].values[0]
        pre_close_new_t = df_change_new.loc[df_change_new[feature] == t_dom_contract].pre_close.values[0]
        pre_settlement_new_t = df_change_new.loc[df_change_new[feature] == t_dom_contract].pre_settlement.values[0]
        
        # 更新因子
        f_actorclose = f_actorclose * (close_old_t_1 / pre_close_new_t)
        f_actorsettlement = f_actorsettlement * (settlement_old_t_1 / pre_settlement_new_t)
    #更新原表df 
    output_df = df.merge(df_change_old.loc[:, [feature,'factor_close', 'factor_settlement']], how = 'left', on = feature)
    
    #新增字段dom_subdom 
    if feature == 'dom_contract':
        output_df['dom_subdom'] = 'DOM'
    else: 
        output_df['dom_subdom'] = 'SUBDOM'
    
    # 整理
    output_df = output_df[['date', 'code', feature ,'close', 'SETTLEMENT', 'OPENINT', 'next_symbol', 'dom_subdom', 'pre_close', 
                          'pre_settlement', 'factor_close', 'factor_settlement']]
    rename_dict = {feature:'symbol', 'SETTLEMENT': 'settlement', 'OPENINT':'open_interest'}
    
    output_df = output_df.rename(columns =rename_dict )
    return output_df

get_factors(part_actorclose_dom, 'dom_contract')



,date,code,symbol,close,settlement,open_interest,next_symbol,dom_subdom,pre_close,pre_settlement,factor_close,factor_settlement
0,2019-12-28,AP,AP2012,NaN,NaN,NaN,None,DOM,7669.0,7700.0,NaN,NaN
1,2019-12-27,AP,AP2012,7669.0,7700.0,4698.0,None,DOM,7706.0,7694.0,NaN,NaN
2,2019-12-26,AP,AP2012,7706.0,7694.0,4686.0,AP2012,DOM,7674.0,7664.0,NaN,NaN
3,2019-12-21,AP,AP2012,NaN,NaN,NaN,None,DOM,7736.0,7755.0,NaN,NaN
4,2019-12-20,AP,AP2012,7736.0,7755.0,3000.0,None,DOM,7962.0,7907.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
248,2018-01-06,AP,AP1812,NaN,NaN,NaN,None,DOM,7230.0,7225.0,1.083656,1.038766
249,2018-01-05,AP,AP1812,7230.0,7225.0,308.0,None,DOM,7200.0,7191.0,1.083656,1.038766
250,2017-12-30,AP,AP1812,NaN,NaN,NaN,None,DOM,7179.0,7177.0,1.083656,1.038766
251,2017-12-29,AP,AP1812,7179.0,7177.0,450.0,None,DOM,7200.0,7145.0,1.083656,1.038766


### 次主力合约

In [21]:
part_dom = get_subdom('AP')# 获得主力合约
#从rawdata获得信息并且合并
part_actorclose_subdom = part_dom.merge(rawdata[['DATE','CONTRACT','OPENINT','SETTLEMENT','symbol', 'close', ]], how = 'left', left_on = ['date','subdom_contract'], right_on = ['DATE', 'CONTRACT'])

    
# 新增三个字段
part_actorclose_subdom['pre_close'] = part_actorclose_subdom.apply(lambda x: get_pre_close(x, 'subdom_contract'), axis = 1) 
part_actorclose_subdom['pre_settlement'] =  part_actorclose_subdom.apply(lambda x: get_pre_settlement(x, 'subdom_contract'), axis = 1) 
part_actorclose_subdom['next_symbol'] =  part_actorclose_subdom.apply(lambda x: get_next_symbol(x, 'subdom_contract'), axis = 1) 

get_factors(part_actorclose_subdom, 'subdom_contract')


/tmp/ipykernel_2449/3604058620.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  change_dom['dom_contract'] = change_dom.apply(lambda x: get_max_openint2(x, 'max_openint'), axis = 1)
/tmp/ipykernel_2449/1973389626.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  switchdom_df['subdom_contract'] = switchdom_df.apply(lambda x: get_max_openint3(x, 'dom_contract'), axis = 1)
/data/envir/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1684: SettingWithCopyWarning: 
A value is trying to be set

,date,code,symbol,close,settlement,open_interest,next_symbol,dom_subdom,pre_close,pre_settlement,factor_close,factor_settlement
0,2019-12-19,AP,AP2005,8110.0,8104.0,100608.0,AP2005,SUBDOM,8095.0,8186.0,NaN,NaN
1,2019-12-19,AP,AP2005,8110.0,8104.0,100608.0,AP2005,SUBDOM,8095.0,8186.0,NaN,NaN
2,2019-12-19,AP,AP2005,8110.0,8104.0,100608.0,AP2005,SUBDOM,8095.0,8186.0,NaN,NaN
3,2019-12-19,AP,AP2005,8110.0,8104.0,100608.0,AP2005,SUBDOM,8095.0,8186.0,NaN,NaN
4,2019-12-19,AP,AP2005,8110.0,8104.0,100608.0,AP2005,SUBDOM,8095.0,8186.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
248,2017-12-28,AP,AP1805,8104.0,8115.0,15558.0,AP1805,SUBDOM,8086.0,8087.0,1.0,1.0
249,2017-12-28,AP,AP1805,8104.0,8115.0,15558.0,AP1805,SUBDOM,8086.0,8087.0,1.0,1.0
250,2017-12-28,AP,AP1805,8104.0,8115.0,15558.0,AP1805,SUBDOM,8086.0,8087.0,1.0,1.0
251,2017-12-28,AP,AP1805,8104.0,8115.0,15558.0,AP1805,SUBDOM,8086.0,8087.0,1.0,1.0


脚本汇总

In [22]:
def contract_factors(code): 
    '''获得该期货的所有信息，包括主力合约复合因子等
    1. 主力合约
    2. 次主力合约
    分开计算'''
    # 主力合约
    part_dom = get_dom(code)# 获得主力合约
    #从rawdata获得信息并且合并
    part_actorclose_dom = part_dom.merge(rawdata[['DATE','CONTRACT','OPENINT','SETTLEMENT','symbol', 'close', ]], how = 'left', left_on = ['date','dom_contract'], right_on = ['DATE', 'CONTRACT'])
    
    def get_pre_close(x,feature):
        # 上一个交易日日期
        pre_date = (datetime.strptime(x.date, "%Y-%m-%d") - timedelta(days = 1)).strftime("%Y-%m-%d")
        part_data1 = part_data.loc[(part_data.CONTRACT == x[feature])&(part_data.DATE == pre_date)]

        if not part_data1.empty: 
            return part_data1.close.values[0]
    def get_pre_settlement(x, feature):
        # 上一个交易日日期
        pre_date = (datetime.strptime(x.date, "%Y-%m-%d") - timedelta(days = 1)).strftime("%Y-%m-%d")
        part_data1 = part_data.loc[(part_data.CONTRACT == x[feature])&(part_data.DATE == pre_date)]

        if not part_data1.empty: 
            return part_data1.SETTLEMENT.values[0]

    def get_next_symbol(x, feature):
        # 下一个交易日期
        next_date = (datetime.strptime(x.date, "%Y-%m-%d") + timedelta(days = 1)).strftime("%Y-%m-%d")
        part_data2 = part_data.loc[(part_data.CONTRACT == x[feature])&(part_data.DATE == next_date)]

        if not part_data2.empty: 
            return part_data2.symbol.values[0]

    # 新增三个字段
    part_actorclose_dom['pre_close'] = part_actorclose_dom.apply(lambda x: get_pre_close(x, 'dom_contract'), axis = 1) 
    part_actorclose_dom['pre_settlement'] =  part_actorclose_dom.apply(lambda x: get_pre_settlement(x, 'dom_contract'), axis = 1) 
    part_actorclose_dom['next_symbol'] =  part_actorclose_dom.apply(lambda x: get_next_symbol(x, 'dom_contract'), axis = 1) 
    
    def get_factors(df, feature):
        '''计算复权因子factor_close & factor_settlement， 计算方式一致'''
        df_change_old = df.drop_duplicates(subset = feature, keep = 'first') #旧主力的最后一个交易日
        df_change_new = df.drop_duplicates(subset = feature, keep = 'last') # 新主力的第一个交易日

        # 初始复权因子 = 1
        f_actorclose = 1
        f_actorsettlement = 1
        for i in range(len(list(df_change_old.index))-1, 0, -1):
            # 首先更新呢表格
            t_1_index = list(df_change_old.index)[i]
            df_change_old.loc[df_change_old.index == t_1_index, 'factor_close'] = f_actorclose
            df_change_old.loc[df_change_old.index == t_1_index, 'factor_settlement'] = f_actorsettlement
            #旧主力合约t-1天（前一天）的收盘价
            close_old_t_1 = df_change_old.loc[df_change_old.index == t_1_index].pre_close.values[0]
            settlement_old_t_1 = df_change_old.loc[df_change_old.index == t_1_index].pre_settlement.values[0]
            # 表示新主力合约t天（当天）的上一个交易日收盘价
            t_index = list(df_change_old.index)[i-1]
            t_dom_contract = df_change_old.loc[df_change_old.index == t_index][feature].values[0]
            pre_close_new_t = df_change_new.loc[df_change_new[feature] == t_dom_contract].pre_close.values[0]
            pre_settlement_new_t = df_change_new.loc[df_change_new[feature] == t_dom_contract].pre_settlement.values[0]

            # 更新因子
            f_actorclose = f_actorclose * (close_old_t_1 / pre_close_new_t)
            f_actorsettlement = f_actorsettlement * (settlement_old_t_1 / pre_settlement_new_t)
        #更新原表df 
        output_df = df.merge(df_change_old.loc[:, [feature,'factor_close', 'factor_settlement']], how = 'left', on = feature)

        #新增字段dom_subdom 
        if feature == 'dom_contract':
            output_df['dom_subdom'] = 'DOM'
        else: 
            output_df['dom_subdom'] = 'SUBDOM'

        # 整理
        output_df = output_df[['date', 'code', feature,'close', 'SETTLEMENT', 'OPENINT', 'next_symbol', 'dom_subdom', 'pre_close', 
                              'pre_settlement', 'factor_close', 'factor_settlement']]
        rename_dict = {feature:'symbol', 'SETTLEMENT': 'settlement', 'OPENINT':'open_interest'}

        output_df = output_df.rename(columns =rename_dict )
        return output_df
    
    dom_contract_info = get_factors(part_actorclose_dom, 'dom_contract') #主合约
    
    #次主力合约
    part_dom = get_subdom(code)# 获得主力合约
    #从rawdata获得信息并且合并
    part_actorclose_subdom = part_dom.merge(rawdata[['DATE','CONTRACT','OPENINT','SETTLEMENT','symbol', 'close', ]], how = 'left', left_on = ['date','subdom_contract'], right_on = ['DATE', 'CONTRACT'])

    # 新增三个字段
    part_actorclose_subdom['pre_close'] = part_actorclose_subdom.apply(lambda x: get_pre_close(x, 'subdom_contract'), axis = 1) 
    part_actorclose_subdom['pre_settlement'] =  part_actorclose_subdom.apply(lambda x: get_pre_settlement(x, 'subdom_contract'), axis = 1) 
    part_actorclose_subdom['next_symbol'] =  part_actorclose_subdom.apply(lambda x: get_next_symbol(x, 'subdom_contract'), axis = 1) 

    subdom_contract_info = get_factors(part_actorclose_subdom, 'subdom_contract')
    
    all_contract_info = dom_contract_info.append(subdom_contract_info, ignore_index = True)
    return all_contract_info
    
contract_factors('AP')    

/tmp/ipykernel_2449/3604058620.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  change_dom['dom_contract'] = change_dom.apply(lambda x: get_max_openint2(x, 'max_openint'), axis = 1)
/data/envir/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1684: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)
/data/envir/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a

,date,code,symbol,close,settlement,open_interest,next_symbol,dom_subdom,pre_close,pre_settlement,factor_close,factor_settlement
0,2019-12-28,AP,AP2012,NaN,NaN,NaN,None,DOM,7669.0,7700.0,NaN,NaN
1,2019-12-27,AP,AP2012,7669.0,7700.0,4698.0,None,DOM,7706.0,7694.0,NaN,NaN
2,2019-12-26,AP,AP2012,7706.0,7694.0,4686.0,AP2012,DOM,7674.0,7664.0,NaN,NaN
3,2019-12-21,AP,AP2012,NaN,NaN,NaN,None,DOM,7736.0,7755.0,NaN,NaN
4,2019-12-20,AP,AP2012,7736.0,7755.0,3000.0,None,DOM,7962.0,7907.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
501,2017-12-28,AP,AP1805,8104.0,8115.0,15558.0,AP1805,SUBDOM,8086.0,8087.0,1.0,1.0
502,2017-12-28,AP,AP1805,8104.0,8115.0,15558.0,AP1805,SUBDOM,8086.0,8087.0,1.0,1.0
503,2017-12-28,AP,AP1805,8104.0,8115.0,15558.0,AP1805,SUBDOM,8086.0,8087.0,1.0,1.0
504,2017-12-28,AP,AP1805,8104.0,8115.0,15558.0,AP1805,SUBDOM,8086.0,8087.0,1.0,1.0


# 总脚本

In [85]:
import pandas as pd
from datetime import datetime, timedelta

test_rawdata = pd.read_csv('test.csv') 
futures_rawdata = pd.read_csv('futures_description.csv')
# 合并
rawdata = test_rawdata.merge(futures_rawdata, how = 'inner', left_on = 'CONTRACT', right_on = 'symbol').sort_values(by = 'DATE',ascending=False)

code_list = list(rawdata.code.unique()) #所有需要遍历的品种

def get_info(code):
    '''1. 获得每个品种的主力和次力合约'''
    def get_dom(code):
        '''对每个期货品种，计算主力合约'''
        part_data = rawdata.loc[rawdata.code == code]
        starttime, endtime = part_data.sort_values(by = 'DATE').iloc[0,3], part_data.sort_values(by = 'DATE').iloc[-1,3] #这个期货品种的起始时间和结束时间

        # 按照开始时间和结束时间，创建时间列表。
        day_gap = (datetime.strptime(endtime, "%Y-%m-%d") - datetime.strptime(starttime, "%Y-%m-%d")).days
        day_list = [(datetime.strptime(endtime, "%Y-%m-%d") - timedelta(days = i)).strftime("%Y-%m-%d") for i in range(0,day_gap)]

        # 创建新的dataframe 填充时间列表和期货种类
        part_df = pd.DataFrame(columns = ['date', 'code'])
        part_df['date'] = day_list 
        part_df['code'] = code

        # 接下来得任务是填充part_df 表格，找到每个时间的主力合约次主力合约
        def get_max_openint(x): 
            '''输出持仓量最大的合约'''
            contract = part_data.loc[part_data.DATE == x].sort_values(by = 'OPENINT', ascending=True)
            if not contract.empty : 
                return contract.iloc[0,2]

        part_df['max_openint'] = part_df.date.apply(lambda x: get_max_openint(x))

        #可以看到有许多空值，现在暂时对空值处理。需要在最大持仓量的合约中找到连续三天持仓的合约， 使其在第4个交易日成为主力合约
        max_openint_list = list(part_df.max_openint)

        for i in range(len(max_openint_list), 2, -1):
            j = i-3
            a_list = max_openint_list[j:i]
            if a_list[0] == a_list[1] == a_list[2]: 
                part_df.loc[part_df.index == j-1, 'dom_contract'] = a_list[2]

        # 如果某主力合约在到期日仍然为持仓量最大，在剩余该品种合约中在最近3个交易日内，累计持仓量最大的合约作为下一个交易日的主力合约
        aa = part_data[['CONTRACT', 'maturity_date']].drop_duplicates() #记录到期时间
        part_df2 = part_df.merge(aa, how = 'left', left_on = 'dom_contract', right_on = 'CONTRACT')
        change_dom = part_df2.loc[(part_df2.maturity_date.notnull()) & (part_df2.date == part_df2.maturity_date)] #需要转化主力合约的表格

        def get_max_openint2(x, feature):
            '''3天内累计持仓量最大的合约'''
            starttime, endtime =(datetime.strptime(x.date, "%Y-%m-%d") - timedelta(days = 2)).strftime("%Y-%m-%d"),(datetime.strptime(x.date, "%Y-%m-%d") + timedelta(days = 1)).strftime("%Y-%m-%d")
            # 计算时间内累计最大持仓量的合约
            sum_3day = part_data.loc[(part_data.CONTRACT != x[feature])&(part_data.DATE < endtime)&(part_data.DATE >= starttime)].groupby(by = 'CONTRACT')['OPENINT'].sum().to_frame().reset_index()
            #输出合约名字
            if not sum_3day.empty:
                return sum_3day.sort_values(by = 'OPENINT', ascending=False).iloc[0, 0] 
        
        if not change_dom.empty:  # 有可能没有需要转化主力的合约
            # 更新需要转化主力合约的表格
            change_dom['dom_contract'] = change_dom.apply(lambda x: get_max_openint2(x, 'max_openint'), axis = 1)

            #更新part_df应该输出的表格中的主力合约
            part_df.loc[part_df.index.isin(list(change_dom.index+1)), 'dom_contract'] = change_dom.dom_contract

        # 输出主力合约表格
        output_part_df = part_df.loc[part_df.dom_contract.notnull(), ['date', 'code', 'dom_contract']].reset_index(drop = True)

        return output_part_df
    
    def get_subdom(code):
        '''输出次主力合约'''
        output_part_df = get_dom(code)
        # 首先找到转换主力合约的次主力合约
        switchdom_df = output_part_df.drop_duplicates(subset = 'dom_contract', keep = "last")

        def get_max_openint3(x, feature):
            '''发生前三天内累计持仓量最大的合约， 和get_max_openint2 区别是时间不同'''
            starttime, endtime =(datetime.strptime(x.date, "%Y-%m-%d") - timedelta(days = 3)).strftime("%Y-%m-%d"), x.date
            # 计算时间内累计最大持仓量的合约
            sum_3day = part_data.loc[(part_data.CONTRACT != x[feature])&(part_data.DATE < endtime)&(part_data.DATE >= starttime)].groupby(by = 'CONTRACT')['OPENINT'].sum().to_frame().reset_index()
            #输出合约名字
            if not sum_3day.empty:
                return sum_3day.sort_values(by = 'OPENINT', ascending=False).iloc[0, 0] 

        # 添加字段 次主力合约
        switchdom_df['subdom_contract'] = switchdom_df.apply(lambda x: get_max_openint3(x, 'dom_contract'), axis = 1)

        # 更新到 output_part_df 
        output_part_df2 = switchdom_df.merge(output_part_df.dom_contract, how = 'left', on = 'dom_contract')

        return output_part_df2[['date', 'code', 'subdom_contract']]
    
    # 输出合约所有12columns信息
    def contract_factors(code): 
        '''获得该期货的所有信息，包括主力合约复合因子等
        1. 主力合约
        2. 次主力合约
        分开计算'''
        # 主力合约
        part_dom = get_dom(code)# 获得主力合约
        #从rawdata获得信息并且合并
        part_actorclose_dom = part_dom.merge(rawdata[['DATE','CONTRACT','OPENINT','SETTLEMENT','symbol', 'close', ]], how = 'left', left_on = ['date','dom_contract'], right_on = ['DATE', 'CONTRACT'])

        def get_pre_close(x,feature):
            # 上一个交易日日期
            pre_date = (datetime.strptime(x.date, "%Y-%m-%d") - timedelta(days = 1)).strftime("%Y-%m-%d")
            part_data1 = part_data.loc[(part_data.CONTRACT == x[feature])&(part_data.DATE == pre_date)]

            if not part_data1.empty: 
                return part_data1.close.values[0]
        def get_pre_settlement(x, feature):
            # 上一个交易日日期
            pre_date = (datetime.strptime(x.date, "%Y-%m-%d") - timedelta(days = 1)).strftime("%Y-%m-%d")
            part_data1 = part_data.loc[(part_data.CONTRACT == x[feature])&(part_data.DATE == pre_date)]

            if not part_data1.empty: 
                return part_data1.SETTLEMENT.values[0]

        def get_next_symbol(x, feature):
            # 下一个交易日期
            next_date = (datetime.strptime(x.date, "%Y-%m-%d") + timedelta(days = 1)).strftime("%Y-%m-%d")
            part_data2 = part_data.loc[(part_data.CONTRACT == x[feature])&(part_data.DATE == next_date)]

            if not part_data2.empty: 
                return part_data2.symbol.values[0]

        # 新增三个字段
        part_actorclose_dom['pre_close'] = part_actorclose_dom.apply(lambda x: get_pre_close(x, 'dom_contract'), axis = 1) 
        part_actorclose_dom['pre_settlement'] =  part_actorclose_dom.apply(lambda x: get_pre_settlement(x, 'dom_contract'), axis = 1) 
        part_actorclose_dom['next_symbol'] =  part_actorclose_dom.apply(lambda x: get_next_symbol(x, 'dom_contract'), axis = 1) 

        def get_factors(df, feature):
            '''计算复权因子factor_close & factor_settlement， 计算方式一致'''

            # 初始复权因子 = 1
            f_actorclose = 1
            f_actorsettlement = 1
            contract_i = df.loc[df.index == len(list(df.index)) -1 ][feature].values[0]
            for i in range(len(list(df.index)) -1 , 0, -1):
                j = i-1
                contract_j = df.loc[df.index == j][feature].values[0] 
                # 首先更新表格
                df.loc[df.index == i, 'factor_close'] = f_actorclose
                df.loc[df.index == i, 'factor_settlement'] = f_actorsettlement

                # 如果下一个交易日主力合约不变，则因子不变
                if contract_j != contract_i:
                    #旧主力合约t-1天（前一天）的收盘价
                    close_old_t_1 = df.loc[df.index == i].close.values[0]
                    settlement_old_t_1 = df.loc[df.index == i].SETTLEMENT.values[0]

                    # 表示新主力合约t天（当天）的上一个交易日收盘价
                    pre_close_new_t = df.loc[df[feature] == contract_j].pre_close.values[0]
                    pre_settlement_new_t = df.loc[df[feature] == contract_j].pre_settlement.values[0]

                    # 更新因子
                    if not (pre_close_new_t is None and pre_settlement_new_t is None ):
                        f_actorclose = f_actorclose * (close_old_t_1 / pre_close_new_t)
                        f_actorsettlement = f_actorsettlement * (settlement_old_t_1 / pre_settlement_new_t)
                # 更新合约
                contract_i = contract_j 
            output_df = df

            #新增字段dom_subdom 
            if feature == 'dom_contract':
                output_df['dom_subdom'] = 'DOM'
            else: 
                output_df['dom_subdom'] = 'SUBDOM'

            # 整理
            output_df = output_df[['date', 'code', feature,'close', 'SETTLEMENT', 'OPENINT', 'next_symbol', 'dom_subdom', 'pre_close', 
                                  'pre_settlement', 'factor_close', 'factor_settlement']]
            rename_dict = {feature:'symbol', 'SETTLEMENT': 'settlement', 'OPENINT':'open_interest'}

            output_df = output_df.rename(columns =rename_dict )
            return output_df

        dom_contract_info = get_factors(part_actorclose_dom, 'dom_contract') #主合约

        #次主力合约
        part_dom = get_subdom(code)# 获得主力合约
        #从rawdata获得信息并且合并
        part_actorclose_subdom = part_dom.merge(rawdata[['DATE','CONTRACT','OPENINT','SETTLEMENT','symbol', 'close', ]], how = 'left', left_on = ['date','subdom_contract'], right_on = ['DATE', 'CONTRACT'])

        # 新增三个字段
        part_actorclose_subdom['pre_close'] = part_actorclose_subdom.apply(lambda x: get_pre_close(x, 'subdom_contract'), axis = 1) 
        part_actorclose_subdom['pre_settlement'] =  part_actorclose_subdom.apply(lambda x: get_pre_settlement(x, 'subdom_contract'), axis = 1) 
        part_actorclose_subdom['next_symbol'] =  part_actorclose_subdom.apply(lambda x: get_next_symbol(x, 'subdom_contract'), axis = 1) 

        subdom_contract_info = get_factors(part_actorclose_subdom, 'subdom_contract')

        all_contract_info = dom_contract_info.append(subdom_contract_info, ignore_index = True)
        
        return all_contract_info
    
    
    return contract_factors(code)
    


In [89]:
# 输出所有的期货品种每天的信息
# 首先输出第一种
type_info = get_info(code_list[0])
for i in range(1, len(code_list)):
    code_i = code_list[i]
    print(code_i)
    type_info = type_info.append(get_info(code_i), ignore_index = True)

/tmp/ipykernel_2449/1504358520.py:87: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  switchdom_df['subdom_contract'] = switchdom_df.apply(lambda x: get_max_openint3(x, 'dom_contract'), axis = 1)


OI


/tmp/ipykernel_2449/1504358520.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  change_dom['dom_contract'] = change_dom.apply(lambda x: get_max_openint2(x, 'max_openint'), axis = 1)


ZC
RM
AP
TA
IF
JR
WH
SF
PM
UR
CY
SR
SA
FG
CJ
MA
SM
T
TF
TS
IC
RS
LR
RI
CF
TC
ME
ER
WS
RO
WT


In [93]:
type_info.to_csv('test_result.csv', index = False )